Import relevant libraries 

In [5]:
import pandas as pd
import numpy as np
import theano
import theano.tensor as t


# Import and Split the Data

In [ ]:
dataset = pd.read_csv("auto-mpg.csv", na_values="?")
dataset.dropna(inplace=True)
dataset = dataset.set_index(['name'])
print dataset.head()


In [ ]:
print dataset.shape

In [ ]:
train = dataset.values[0:250, :]
print train.shape
print train[:10, 0]
test = dataset.values[250:, :]

In [ ]:
train_x = train[:, 1:]
train_y = train[:, 0]
test_x = test[:, 1:]
test_y = test[:, 0]
print train_y[:10]


# Set up the theano variables 

In [4]:
x = t.matrix("X")
y = t.vector("y")

w_value = numpy.zeros(7, dtype=theano.config.floatX)
b_value = numpy.asarray(0., dtype=theano.config.floatX)


w = theano.shared(value=w_value, name="weights")
b = theano.shared(value=b_value, name="bias")
params = [w, b]

NameError: name 't' is not defined

## Build our functions 

In [ ]:
# Build Hypothesis
lin_hypothesis = t.dot(w.T, x) + b

# Define objective function
mean_sq_error = t.mean((lin_hypothesis - y)**2)

# Get the gradient 
gw, gb = t.grad(mean_sq_error, wrt=[w, b])

# Define the learning rate 
alpha = 0.01

# Define the update 
updates = (
    (w, w - alpha * gw), 
    (b, b - alpha * gb)
)

train_function = theano.function(inputs=[x, y], outputs=[lin_hypothesis, mean_sq_error], updates=updates)

test_function = theano.function(inputs=[x, y], outputs=[lin_hypothesis])

## Train our Model 

In [1]:
for i in range(50):
    h, error = train_function([train_x, train_y])
    print error

test_error = test_function([test_x, test_y])

NameError: name 'train_function' is not defined

# MLP 
## Set up Theano Variables  

In [ ]:
x = t.matrix("X")
y = t.vector("y")

fan_in = 7
n_nodes = 4

bound = np.sqrt(6. / (fan_in + num_nodes))

w1_value = np.asarray(
            np.random.uniform(
                low=-4*bound,
                high=4*bound,
                size=(fan_in, num_nodes)
            ),
            dtype=theano.config.floatX
        )
b1_value = np.zeros(shape=num_nodes, dtype=theano.config.floatX)

w2_value = numpy.zeros(4, dtype=theano.config.floatX)
b2_value = numpy.asarray(0., dtype=theano.config.floatX)


w1 = theano.shared(value=w1_value, name="weights1")
b1 = theano.shared(value=b1_value, name="bias1")

w2 = theano.shared(value=w1_value, name="weights1")
b2 = theano.shared(value=b1_value, name="bias1")

params = [w1, b1, w2, b2]

In [3]:
# lin activation 
z = t.dot(w1.T, x) + b
# non linear logistic activation - sigmoid i.e. activations/probabilites 
a = 1 / (1 + t.exp(-z)) 
# get linear output 
y_hat_hypothesis = t.dot(w.T, a) + b 

# Define objective function
mean_sq_error = t.mean(t.sq((y_hat_hypothesis - y))

# Get the gradient 
gw1, gb1, gw2, gb2 = t.grad(mean_sq_error, wrt=[w1, b1, w2, b2])

# Define the learning rate 
alpha = 0.01

# Define the update 
updates = (
    (w1, w1 - alpha * gw1), 
    (b1, b1 - alpha * gb1),
    (w2, w2 - alpha * gw2), 
    (b2, b2 - alpha * gb2)
)

train_function = theano.function(inputs=[x, y], outputs=[y_hat_hypothesis, mean_sq_error], updates=updates)

test_function = theano.function(inputs=[x, y], outputs=[lin_hypothesis])


SyntaxError: invalid syntax (<ipython-input-3-7f934c898e43>, line 12)